# 토픽 모델링(Topic Modeling)

## [1] TF-IDF(Term Frequency-Inverse Document Frequency) : 단어 빈도-역 문서 빈도

In [1]:
import pandas as pd # 데이터프레임 사용을 위해
import numpy as np
from math import log # IDF 계산을 위해

In [2]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [3]:
N = len(docs) # 총 문서의 수

def tf(t, d):  # 특정 문서 d에서의 특정 단어 t의 등장 횟수
    return d.count(t)

def idf(t):  # df(t):특정 단어 t가 등장한 문서의 수,idf(t):df(t)의 역수, log는 N이 커지면 값이 기하급수로 늘어나는걸 방지,분모 1은 df가 0일 경우 대비
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d): # tf(t,d)와 idf(t)를 곱한 값
    return tf(t,d)* idf(t)

In [4]:
# Document-Term Matrix(DTM) : 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [5]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [6]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 사이킷런을 이용한 DTM과 TF-IDF

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


## [2] 잠재 의미 분석(Latent Semantic Analysis, LSA)

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
# wenty Newsgroups 뉴스 그룹 데이터 읽어오기 
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data   # list
len(documents) 

11314

In [11]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [12]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 텍스트 전처리

In [13]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [14]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거

In [16]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


### TF-IDF 행렬 만들기

In [17]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [18]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

### 토픽 모델링(Topic Modeling)

In [20]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [21]:
np.shape(svd_model.components_)

(20, 1000)

In [22]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

# 값이 큰 5개의 값을 찾아서 단어로 출력
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21378), ('know', 0.20032), ('people', 0.19317), ('think', 0.17807), ('good', 0.15105)]
Topic 2: [('thanks', 0.3292), ('windows', 0.29094), ('card', 0.18016), ('drive', 0.1739), ('mail', 0.15126)]
Topic 3: [('game', 0.37161), ('team', 0.32553), ('year', 0.28204), ('games', 0.25416), ('season', 0.18464)]
Topic 4: [('drive', 0.52804), ('scsi', 0.20035), ('disk', 0.15514), ('hard', 0.15506), ('card', 0.14041)]
Topic 5: [('windows', 0.4052), ('file', 0.25609), ('window', 0.18057), ('files', 0.1619), ('program', 0.1401)]
Topic 6: [('chip', 0.16138), ('government', 0.16026), ('mail', 0.15625), ('space', 0.1505), ('information', 0.13575)]
Topic 7: [('like', 0.67109), ('bike', 0.14268), ('know', 0.11428), ('chip', 0.11058), ('sounds', 0.10398)]
Topic 8: [('card', 0.45115), ('sale', 0.21607), ('video', 0.21391), ('monitor', 0.14913), ('offer', 0.14872)]
Topic 9: [('know', 0.44979), ('card', 0.35491), ('chip', 0.17206), ('video', 0.15184), ('government', 0.15053)]
Topic 10: [

In [23]:
documents[20]

"\n\n\n\n\tI'd like to see this info as well.  As for wavelength, I think\nyou're primarily going to find two - 880 nM +/- a bit, and/or 950 nM\n+/- a bit.  Usually it is about 10 nM either way.  The two most common\nI have seen were 880 and 950 but I have also heard of 890 and 940.\nI'm not sure that the 10 nM one way or another will make a great deal of\ndifference.\n\n\tAnother suggestion - find a brand of TV that uses an IR remote,\nand go look at the SAMS photofact for it.  You can often find some very\ndetailed schematics and parts list for not only the receiver but the\ntransmitter as well, including carrier freq. specs. and tone decoding\nspecs. if the system uses that."

## [3] 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)]

In [24]:
# ! pip install gensim
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [25]:
print(dictionary[66])

faith


In [26]:
len(dictionary)

64281

#### LDA 모델 훈련시키기

In [27]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.018*"game" + 0.016*"team" + 0.012*"year" + 0.012*"games"')
(1, '0.011*"file" + 0.011*"available" + 0.011*"files" + 0.009*"window"')
(2, '0.014*"would" + 0.012*"people" + 0.009*"think" + 0.007*"like"')
(3, '0.022*"food" + 0.019*"tobacco" + 0.015*"radar" + 0.010*"detector"')
(4, '0.023*"jesus" + 0.015*"christian" + 0.013*"bible" + 0.012*"church"')
(5, '0.016*"drive" + 0.015*"thanks" + 0.012*"card" + 0.011*"please"')
(6, '0.011*"people" + 0.010*"state" + 0.009*"government" + 0.008*"rights"')
(7, '0.045*"space" + 0.016*"nasa" + 0.015*"data" + 0.009*"printf"')
(8, '0.007*"chip" + 0.007*"used" + 0.006*"system" + 0.006*"encryption"')
(9, '0.041*"file" + 0.038*"output" + 0.032*"entry" + 0.020*"program"')
(10, '0.018*"mail" + 0.015*"information" + 0.013*"send" + 0.013*"list"')
(11, '0.027*"period" + 0.015*"power" + 0.014*"filename" + 0.012*"play"')
(12, '0.016*"israel" + 0.013*"said" + 0.010*"israeli" + 0.010*"armenians"')
(13, '0.010*"cluster" + 0.009*"offset" + 0.008*"init" + 0.008*"ec

In [28]:
print(ldamodel.print_topics())

[(0, '0.018*"game" + 0.016*"team" + 0.012*"year" + 0.012*"games" + 0.010*"season" + 0.008*"play" + 0.008*"players" + 0.008*"league" + 0.008*"hockey" + 0.007*"last"'), (1, '0.011*"file" + 0.011*"available" + 0.011*"files" + 0.009*"window" + 0.009*"program" + 0.009*"windows" + 0.009*"version" + 0.008*"software" + 0.008*"image" + 0.008*"server"'), (2, '0.014*"would" + 0.012*"people" + 0.009*"think" + 0.007*"like" + 0.007*"many" + 0.007*"even" + 0.006*"know" + 0.006*"well" + 0.005*"question" + 0.005*"believe"'), (3, '0.022*"food" + 0.019*"tobacco" + 0.015*"radar" + 0.010*"detector" + 0.009*"navy" + 0.008*"eating" + 0.007*"foods" + 0.006*"detectors" + 0.006*"astros" + 0.006*"robert"'), (4, '0.023*"jesus" + 0.015*"christian" + 0.013*"bible" + 0.012*"church" + 0.011*"christians" + 0.010*"faith" + 0.010*"christ" + 0.006*"also" + 0.006*"believe" + 0.006*"christianity"'), (5, '0.016*"drive" + 0.015*"thanks" + 0.012*"card" + 0.011*"please" + 0.010*"anyone" + 0.010*"know" + 0.010*"would" + 0.009*"

#### 시각화

In [29]:
# ! pip install pyLDAvis

In [30]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

####  문서 별 토픽 분포 보기

In [31]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.43362376), (12, 0.11234042), (16, 0.28375313), (18, 0.15736765)]
1 번째 문서의 topic 비율은 [(2, 0.39014995), (4, 0.06535141), (8, 0.037178494), (19, 0.48623967)]
2 번째 문서의 topic 비율은 [(2, 0.39712596), (6, 0.022838457), (8, 0.11906457), (12, 0.31123307), (19, 0.13757186)]
3 번째 문서의 topic 비율은 [(8, 0.65977335), (19, 0.32591808)]
4 번째 문서의 topic 비율은 [(0, 0.5759439), (9, 0.07839137), (19, 0.3141743)]


In [32]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [33]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2.0,0.4336,"[(2, 0.43361208), (12, 0.11234203), (16, 0.283..."
1,1,19.0,0.4862,"[(2, 0.39018586), (4, 0.06535015), (8, 0.03718..."
2,2,2.0,0.3971,"[(2, 0.39709482), (6, 0.02283877), (8, 0.11906..."
3,3,8.0,0.6597,"[(8, 0.65972567), (19, 0.3259658)]"
4,4,0.0,0.5759,"[(0, 0.5758711), (9, 0.07837416), (19, 0.31426..."
5,5,4.0,0.3672,"[(2, 0.33257926), (4, 0.36715406), (6, 0.14956..."
6,6,5.0,0.3389,"[(0, 0.062628195), (1, 0.115001656), (2, 0.285..."
7,7,2.0,0.5966,"[(2, 0.59655726), (8, 0.047016833), (12, 0.233..."
8,8,19.0,0.4999,"[(0, 0.18634029), (1, 0.11597463), (10, 0.0368..."
9,9,8.0,0.5115,"[(2, 0.04279843), (5, 0.08068991), (8, 0.51151..."


###  잠재 디리클레 할당(LDA) 구현 : 뉴스 기사 헤드라인 

In [34]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [35]:
print(len(data))

1082168


In [36]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [37]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [38]:
# 텍스트 전처리
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


In [40]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [42]:
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


#### TF-IDF 행렬 만들기

In [44]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [45]:
text['headline_text'][:5]

0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(text['headline_text'])
X.shape # TF-IDF 행렬의 크기 확인

(1082168, 1000)

####  토픽 모델링

In [47]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [48]:
lda_top = lda_model.fit_transform(X)

In [49]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[[1.00001533e-01 1.00001269e-01 1.00004179e-01 ... 1.00006124e-01
  1.00003111e-01 1.00003064e-01]
 [1.00001199e-01 1.13513398e+03 3.50170830e+03 ... 1.00009349e-01
  1.00001896e-01 1.00002937e-01]
 [1.00001811e-01 1.00001151e-01 1.00003566e-01 ... 1.00002693e-01
  1.00002061e-01 7.53381835e+02]
 ...
 [1.00001065e-01 1.00001689e-01 1.00003278e-01 ... 1.00006721e-01
  1.00004902e-01 1.00004759e-01]
 [1.00002401e-01 1.00000732e-01 1.00002989e-01 ... 1.00003517e-01
  1.00001428e-01 1.00005266e-01]
 [1.00003427e-01 1.00002313e-01 1.00007340e-01 ... 1.00003732e-01
  1.00001207e-01 1.00005153e-01]]
(10, 1000)


In [50]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
get_topics(lda_model.components_,terms)

Topic 1: [('government', 8725.19), ('sydney', 8393.29), ('queensland', 7720.12), ('change', 5874.27), ('home', 5674.38)]
Topic 2: [('australia', 13691.08), ('australian', 11088.95), ('melbourne', 7528.43), ('world', 6707.7), ('south', 6677.03)]
Topic 3: [('death', 5935.06), ('interview', 5924.98), ('kill', 5851.6), ('jail', 4632.85), ('life', 4275.27)]
Topic 4: [('house', 6113.49), ('2016', 5488.19), ('state', 4923.41), ('brisbane', 4857.21), ('tasmania', 4610.97)]
Topic 5: [('court', 7542.74), ('attack', 6959.64), ('open', 5663.0), ('face', 5193.63), ('warn', 5115.01)]
Topic 6: [('market', 5545.86), ('rural', 5502.89), ('plan', 4828.71), ('indigenous', 4223.4), ('power', 3968.26)]
Topic 7: [('charge', 8428.8), ('election', 7561.63), ('adelaide', 6758.36), ('make', 5658.99), ('test', 5062.69)]
Topic 8: [('police', 12092.44), ('crash', 5281.14), ('drug', 4290.87), ('beat', 3257.58), ('rise', 2934.92)]
Topic 9: [('fund', 4693.03), ('labor', 4047.69), ('national', 4038.68), ('council', 40